## Trabalho Final

Este notebook tem como objetivo realizar o tratamento e análise exploratória dos dados eleitorais disponibilizados pelo TSE. 

### Etapas:
1. Carregamento e inspeção inicial dos dados.
2. Tratamento de inconsistências e valores faltantes.
3. Integração das bases de dados.
4. Análise exploratória e visualizações.
5. Preparação do dataset final para análise.

## 1. Carregamento e inspeção inicial dos dados.

In [997]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import chardet

In [998]:
# Lista com os caminhos dos arquivos CSV
arquivos_csv = [
    "candidatos\\consulta_cand_2012_RS.csv",
    "candidatos\\consulta_cand_2016_RS.csv",
    "candidatos\\consulta_cand_2020_RS.csv",
    "resultados\\votacao_candidato_munzona_2012_RS.csv",
    "resultados\\votacao_candidato_munzona_2016_RS.csv",
    "resultados\\votacao_candidato_munzona_2020_RS.csv"
]

# Função para transformar um arquivo CSV em UTF-8
def transformar_para_utf8(caminho_arquivo):
    # Detectar a codificação do arquivo
    with open(caminho_arquivo, "rb") as f:
        result = chardet.detect(f.read(10000))
        encoding_detectada = result['encoding']

    # Ler o arquivo com a codificação detectada
    with open(caminho_arquivo, "r", encoding=encoding_detectada, errors="replace") as infile:
        conteudo = infile.read()

    # Sobrescrever o arquivo em UTF-8
    with open(caminho_arquivo, "w", encoding="utf-8") as outfile:
        outfile.write(conteudo)

# Transformar todos os arquivos na lista
for arquivo in arquivos_csv:
    transformar_para_utf8(arquivo)

print("Todos os arquivos foram transformados para UTF-8 com sucesso.")

Todos os arquivos foram transformados para UTF-8 com sucesso.


In [999]:
# Definindo os caminhos dos arquivos
path_cand_2012 = "candidatos\\consulta_cand_2012_RS.csv"
path_cand_2016 = "candidatos\\consulta_cand_2016_RS.csv"
path_cand_2020 = "candidatos\\consulta_cand_2020_RS.csv"

path_votos_2012 = "resultados\\votacao_candidato_munzona_2012_RS.csv"
path_votos_2016 = "resultados\\votacao_candidato_munzona_2016_RS.csv"
path_votos_2020 = "resultados\\votacao_candidato_munzona_2020_RS.csv"

# Carregando os dados
cand_2012 = pd.read_csv(path_cand_2012, sep=";", encoding="utf-8")
cand_2016 = pd.read_csv(path_cand_2016, sep=";", encoding="utf-8")
cand_2020 = pd.read_csv(path_cand_2020, sep=";", encoding="utf-8")

votos_2012 = pd.read_csv(path_votos_2012, sep=";", encoding="utf-8")
votos_2016 = pd.read_csv(path_votos_2016, sep=";", encoding="utf-8")
votos_2020 = pd.read_csv(path_votos_2020, sep=";", encoding="utf-8")

C:\Users\alicc\AppData\Local\Temp\ipykernel_4172\3507650244.py:11: DtypeWarning: Columns (57) have mixed types. Specify dtype option on import or set low_memory=False.
  cand_2012 = pd.read_csv(path_cand_2012, sep=";", encoding="utf-8")


In [1000]:
# # Exibindo as primeiras linhas de cada dataset para inspeção inicial

# print("Candidatos 2012:")
# display(cand_2012.head())
# len(cand_2012)

# print("\nCandidatos 2016:")
# display(cand_2016.head())
# len(cand_2016)

# print("\nCandidatos 2020:")
# display(cand_2020.head())
# len(cand_2020)

# print("\nVotação 2012:")
# display(votos_2012.head())
# len(votos_2012)

# print("\nVotação 2016:")
# display(votos_2016.head())
# len(votos_2016)

# print("\nVotação 2020:")
# display(votos_2020.head())
# len(votos_2020)

In [1001]:
# Função auxiliar para inspecionar as bases de dados
def resumo_dataset(nome, dataset):
    print(f"Resumo do dataset: {nome}")
    print("-" * 50)
    print(f"Dimensões: {dataset.shape}")
    print("\nTipos de dados:")
    print(dataset.dtypes)
    print("\nDados faltantes:")
    print(dataset.isnull().sum())
    print("\nAmostra:")

# Inspecionando cada dataset
resumo_dataset("Candidatos 2012", cand_2012)
resumo_dataset("Candidatos 2016", cand_2016)
resumo_dataset("Candidatos 2020", cand_2020)

resumo_dataset("Votação 2012", votos_2012)
resumo_dataset("Votação 2016", votos_2016)
resumo_dataset("Votação 2020", votos_2020)

Resumo do dataset: Candidatos 2012
--------------------------------------------------
Dimensões: (29079, 63)

Tipos de dados:
DT_GERACAO                      object
HH_GERACAO                      object
ANO_ELEICAO                      int64
CD_TIPO_ELEICAO                  int64
NM_TIPO_ELEICAO                 object
                                 ...  
CD_SITUACAO_CANDIDATO_PLEITO     int64
DS_SITUACAO_CANDIDATO_PLEITO    object
CD_SITUACAO_CANDIDATO_URNA       int64
DS_SITUACAO_CANDIDATO_URNA      object
ST_CANDIDATO_INSERIDO_URNA      object
Length: 63, dtype: object

Dados faltantes:
DT_GERACAO                      0
HH_GERACAO                      0
ANO_ELEICAO                     0
CD_TIPO_ELEICAO                 0
NM_TIPO_ELEICAO                 0
                               ..
CD_SITUACAO_CANDIDATO_PLEITO    0
DS_SITUACAO_CANDIDATO_PLEITO    0
CD_SITUACAO_CANDIDATO_URNA      0
DS_SITUACAO_CANDIDATO_URNA      0
ST_CANDIDATO_INSERIDO_URNA      0
Length: 63, dtype: int64

A

In [1002]:
# Selecionando colunas relevantes dos datasets de candidatos
colunas_candidatos = [
    "SQ_CANDIDATO",      # Número sequencial da candidato(a)  
    "ANO_ELEICAO",       # Ano da eleição
    "SG_UE",             # Sigla da Unidade Eleitoral
    "NM_UE",             # Nome da Unidade Eleitoral
    "CD_CARGO",          # Código do cargo
    "DS_CARGO",          # Descrição do cargo
    "NM_CANDIDATO",      # Nome completo do candidato
    "CD_GENERO",         # Código do gênero
    "DS_GENERO",         # Descrição do gênero
    "CD_GRAU_INSTRUCAO", # Código do grau de instrução
    "DS_GRAU_INSTRUCAO", # Descrição do grau de instrução
    "CD_COR_RACA",       # Código da cor/raça 1:BRANCA 2:PRETA 3:PARDA 4:AMARELA 5:INDIGENA 6:NAO INFORMADO
    "DS_COR_RACA"        # Descrição da cor/raça da candidata ou candidato
]

cand_2012 = cand_2012[colunas_candidatos]
cand_2016 = cand_2016[colunas_candidatos]
cand_2020 = cand_2020[colunas_candidatos]

# Exibindo as dimensões após a seleção
print(f"Candidatos 2012: {cand_2012.shape}")
print(f"Candidatos 2016: {cand_2016.shape}")
print(f"Candidatos 2020: {cand_2020.shape}")

Candidatos 2012: (29079, 13)
Candidatos 2016: (28934, 13)
Candidatos 2020: (33611, 13)


In [1003]:
# Selecionando colunas relevantes dos datasets de votação
colunas_votacao = [
    "SQ_CANDIDATO",     # Número sequencial da candidato(a) 
    "QT_VOTOS_NOMINAIS" # Quantidade de votos nominais
]                

votos_2012 = votos_2012[colunas_votacao]
votos_2016 = votos_2016[colunas_votacao]
votos_2020 = votos_2020[colunas_votacao]


print(f"Votação 2012: {votos_2012.shape}")
print(f"Votação 2016: {votos_2016.shape}")
print(f"Votação 2020: {votos_2020.shape}")


Votação 2012: (37468, 2)
Votação 2016: (37234, 2)
Votação 2020: (44541, 2)


# 2. Tratamento de inconsistências e valores faltantes.

In [1004]:
# Agrupando dataframes de resultados por candidato e somando seus votos

votos_2012 = votos_2012.groupby("SQ_CANDIDATO", as_index=False)["QT_VOTOS_NOMINAIS"].sum()
votos_2016 = votos_2016.groupby("SQ_CANDIDATO", as_index=False)["QT_VOTOS_NOMINAIS"].sum()
votos_2020 = votos_2020.groupby("SQ_CANDIDATO", as_index=False)["QT_VOTOS_NOMINAIS"].sum()

In [1005]:
# Função para verificar valores nulos em um DataFrame
def tratar_valores_nulos(dataset, nome, colunas_essenciais):
    print(f"Tratando valores nulos no dataset: {nome}")
    print("-" * 50)
    
    # Contando valores nulos por coluna
    print("Quantidade de valores nulos por coluna:")
    print(dataset.isnull().sum())
    
    # Remover linhas com valores nulos nas colunas essenciais
    dataset = dataset.dropna(subset=colunas_essenciais)
    print(f"\nApós remoção de valores nulos, dimensões: {dataset.shape}")
    print("=" * 50, "\n")
    
    return dataset

# Colunas essenciais para os datasets de votação
colunas_essenciais_votacao = ["SQ_CANDIDATO", "QT_VOTOS_NOMINAIS"]

# Colunas essenciais para os datasets de candidatos
colunas_essenciais_candidatos = ["SQ_CANDIDATO", "DS_CARGO", "NM_CANDIDATO", "DS_GRAU_INSTRUCAO"]

# Aplicando a função para os datasets de votação
votos_2012 = tratar_valores_nulos(votos_2012, "Votação 2012", colunas_essenciais_votacao)
votos_2016 = tratar_valores_nulos(votos_2016, "Votação 2016", colunas_essenciais_votacao)
votos_2020 = tratar_valores_nulos(votos_2020, "Votação 2020", colunas_essenciais_votacao)

# Aplicando a função para os datasets de candidatos
cand_2012 = tratar_valores_nulos(cand_2012, "Candidatos 2012", colunas_essenciais_candidatos)
cand_2016 = tratar_valores_nulos(cand_2016, "Candidatos 2016", colunas_essenciais_candidatos)
cand_2020 = tratar_valores_nulos(cand_2020, "Candidatos 2020", colunas_essenciais_candidatos)


Tratando valores nulos no dataset: Votação 2012
--------------------------------------------------
Quantidade de valores nulos por coluna:
SQ_CANDIDATO         0
QT_VOTOS_NOMINAIS    0
dtype: int64

Após remoção de valores nulos, dimensões: (26854, 2)

Tratando valores nulos no dataset: Votação 2016
--------------------------------------------------
Quantidade de valores nulos por coluna:
SQ_CANDIDATO         0
QT_VOTOS_NOMINAIS    0
dtype: int64

Após remoção de valores nulos, dimensões: (26465, 2)

Tratando valores nulos no dataset: Votação 2020
--------------------------------------------------
Quantidade de valores nulos por coluna:
SQ_CANDIDATO         0
QT_VOTOS_NOMINAIS    0
dtype: int64

Após remoção de valores nulos, dimensões: (31108, 2)

Tratando valores nulos no dataset: Candidatos 2012
--------------------------------------------------
Quantidade de valores nulos por coluna:
SQ_CANDIDATO         0
ANO_ELEICAO          0
SG_UE                0
NM_UE                0
CD_CARG

In [1006]:
# Função para verificar e corrigir valores negativos em um DataFrame
def tratar_valores_negativos(dataset, nome, colunas_essenciais):
    print(f"Tratando valores negativos no dataset: {nome}")
    print("-" * 50)
    
    # Verificando valores negativos nas colunas numéricas
    for col in colunas_essenciais:
        if dataset[col].dtype in ['int64', 'float64']:  # Só vamos verificar as colunas numéricas
            negativos = dataset[dataset[col] < 0]
            if not negativos.empty:
                print(f"Valores negativos encontrados na coluna '{col}':")
                print(negativos[[col]])  # Exibe os valores negativos encontrados
            
            # Corrigindo os valores negativos (substituindo por 0)
            dataset[col] = dataset[col].apply(lambda x: max(x, 0))  # Substitui valores negativos por 0

    print(f"\nApós tratamento de valores negativos, dimensões: {dataset.shape}")
    print("=" * 50, "\n")
    
    return dataset

# Colunas essenciais para os datasets de votação (onde esperamos valores numéricos, como votos)
colunas_essenciais_votacao = ["QT_VOTOS_NOMINAIS"]

# Aplicando a função para os datasets de votação
votos_2012 = tratar_valores_negativos(votos_2012, "Votação 2012", colunas_essenciais_votacao)
votos_2016 = tratar_valores_negativos(votos_2016, "Votação 2016", colunas_essenciais_votacao)
votos_2020 = tratar_valores_negativos(votos_2020, "Votação 2020", colunas_essenciais_votacao)

Tratando valores negativos no dataset: Votação 2012
--------------------------------------------------

Após tratamento de valores negativos, dimensões: (26854, 2)

Tratando valores negativos no dataset: Votação 2016
--------------------------------------------------

Após tratamento de valores negativos, dimensões: (26465, 2)

Tratando valores negativos no dataset: Votação 2020
--------------------------------------------------

Após tratamento de valores negativos, dimensões: (31108, 2)



In [1007]:
# Função para transformar todo o DataFrame em maiúsculas
def converter_para_upper(df):
    return df.applymap(lambda x: x.upper() if isinstance(x, str) else x)

# Convertendo todos os DataFrames para maiúsculas
cand_2012 = converter_para_upper(cand_2012)
cand_2016 = converter_para_upper(cand_2016)
cand_2020 = converter_para_upper(cand_2020)

votos_2012 = converter_para_upper(votos_2012)
votos_2016 = converter_para_upper(votos_2016)
votos_2020 = converter_para_upper(votos_2020)

C:\Users\alicc\AppData\Local\Temp\ipykernel_4172\758687939.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(lambda x: x.upper() if isinstance(x, str) else x)


In [1008]:
# Filtrando os dados para o município de Porto Alegre e os cargos Vereador e Prefeito
municipio = "PORTO ALEGRE"
cargos = ["VEREADOR", "PREFEITO"]

# Aplicando os filtros nos datasets de candidatos
cand_2012 = cand_2012[(cand_2012["NM_UE"] == municipio) & (cand_2012["DS_CARGO"].isin(cargos))]
cand_2016 = cand_2016[(cand_2016["NM_UE"] == municipio) & (cand_2016["DS_CARGO"].isin(cargos))]
cand_2020 = cand_2020[(cand_2020["NM_UE"] == municipio) & (cand_2020["DS_CARGO"].isin(cargos))]

# Exibindo as dimensões após o filtro
print(f"Candidatos 2012: {cand_2012.shape}")
print(f"Candidatos 2016: {cand_2016.shape}")
print(f"Candidatos 2020: {cand_2020.shape}")

Candidatos 2012: (607, 13)
Candidatos 2016: (611, 13)
Candidatos 2020: (881, 13)


In [1009]:
# Verificando duplicatas

print(f"Duplicatas em candidatos 2012: {cand_2012.duplicated(subset='SQ_CANDIDATO').sum()}")
print(f"Duplicatas em candidatos 2016: {cand_2016.duplicated(subset='SQ_CANDIDATO').sum()}")
print(f"Duplicatas em candidatos 2020: {cand_2020.duplicated(subset='SQ_CANDIDATO').sum()}")

print(f"Duplicatas em votos 2012: {votos_2012.duplicated(subset=['SQ_CANDIDATO', 'QT_VOTOS_NOMINAIS']).sum()}")
print(f"Duplicatas em votos 2012: {votos_2016.duplicated(subset=['SQ_CANDIDATO', 'QT_VOTOS_NOMINAIS']).sum()}")
print(f"Duplicatas em votos 2012: {votos_2020.duplicated(subset=['SQ_CANDIDATO', 'QT_VOTOS_NOMINAIS']).sum()}")

Duplicatas em candidatos 2012: 0
Duplicatas em candidatos 2016: 2
Duplicatas em candidatos 2020: 2
Duplicatas em votos 2012: 0
Duplicatas em votos 2012: 0
Duplicatas em votos 2012: 0


In [1010]:
# Removendo duplicatas nos datasets de candidatos (baseado na coluna SQ_CANDIDATO)
cand_2012 = cand_2012.drop_duplicates(subset="SQ_CANDIDATO", keep="first")  # Mantém a primeira ocorrência
cand_2016 = cand_2016.drop_duplicates(subset="SQ_CANDIDATO", keep="first")  # Mantém a primeira ocorrência
cand_2020 = cand_2020.drop_duplicates(subset="SQ_CANDIDATO", keep="first")  # Mantém a primeira ocorrência

# Removendo duplicatas nos datasets de candidatos (baseado na coluna SQ_CANDIDATO)
votos_2012 = votos_2012.drop_duplicates(subset="SQ_CANDIDATO", keep="first")  # Mantém a primeira ocorrência
votos_2016 = votos_2016.drop_duplicates(subset="SQ_CANDIDATO", keep="first")  # Mantém a primeira ocorrência
votos_2020 = votos_2020.drop_duplicates(subset="SQ_CANDIDATO", keep="first")  # Mantém a primeira ocorrência

# Exibindo as dimensões após a remoção de duplicatas
print(f"Duplicatas em candidatos 2012: {cand_2012.duplicated(subset='SQ_CANDIDATO').sum()}")
print(f"Duplicatas em candidatos 2016: {cand_2016.duplicated(subset='SQ_CANDIDATO').sum()}")
print(f"Duplicatas em candidatos 2020: {cand_2020.duplicated(subset='SQ_CANDIDATO').sum()}")

print(f"Duplicatas em votos 2012: {votos_2012.duplicated(subset='SQ_CANDIDATO').sum()}")
print(f"Duplicatas em votos 2016: {votos_2016.duplicated(subset='SQ_CANDIDATO').sum()}")
print(f"Duplicatas em votos 2020: {votos_2020.duplicated(subset='SQ_CANDIDATO').sum()}")


Duplicatas em candidatos 2012: 0
Duplicatas em candidatos 2016: 0
Duplicatas em candidatos 2020: 0
Duplicatas em votos 2012: 0
Duplicatas em votos 2016: 0
Duplicatas em votos 2020: 0


In [1011]:
# Certificando que QT_VOTOS_NOMINAIS não contenha valores negativos

votos_2012 = votos_2012[votos_2012["QT_VOTOS_NOMINAIS"] >= 0]
votos_2016 = votos_2016[votos_2016["QT_VOTOS_NOMINAIS"] >= 0]
votos_2020 = votos_2020[votos_2020["QT_VOTOS_NOMINAIS"] >= 0]

In [1012]:
import os

# Criando a estrutura de pastas: dados_tratados/candidatos_tratados e dados_tratados/resultados_tratados
os.makedirs("dados_tratados/candidatos_tratados", exist_ok=True)
os.makedirs("dados_tratados/resultados_tratados", exist_ok=True)

# Salvando os dados de candidatos na subpasta 'candidatos_tratados'
cand_2012.to_csv("dados_tratados/candidatos_tratados/cand_votos_2012_tratado.csv", index=False, encoding="utf-8")
cand_2016.to_csv("dados_tratados/candidatos_tratados/cand_votos_2016_tratado.csv", index=False, encoding="utf-8")
cand_2020.to_csv("dados_tratados/candidatos_tratados/cand_votos_2020_tratado.csv", index=False, encoding="utf-8")

# Salvando os dados de votação na subpasta 'resultados_tratados'
votos_2012.to_csv("dados_tratados/resultados_tratados/votos_2012_tratado.csv", index=False, encoding="utf-8")
votos_2016.to_csv("dados_tratados/resultados_tratados/votos_2016_tratado.csv", index=False, encoding="utf-8")
votos_2020.to_csv("dados_tratados/resultados_tratados/votos_2020_tratado.csv", index=False, encoding="utf-8")

print("Arquivos CSV salvos com sucesso na estrutura 'dados_tratados/candidatos_tratados' e 'dados_tratados/resultados_tratados'!")

Arquivos CSV salvos com sucesso na estrutura 'dados_tratados/candidatos_tratados' e 'dados_tratados/resultados_tratados'!


### 1.1 Integração

In [1013]:
print(f"Dimensões antes integração - 2012: {cand_votos_2012.shape}")
print(f"Dimensões antes integração - 2016: {cand_votos_2016.shape}")
print(f"Dimensões antes integração - 2020: {cand_votos_2020.shape}")

Dimensões antes integração - 2012: (576, 17)
Dimensões antes integração - 2016: (560, 17)
Dimensões antes integração - 2020: (845, 17)


In [1014]:
cand_votos_2012 = cand_2012.merge(votos_2012, on="SQ_CANDIDATO", how="inner")
cand_votos_2016 = cand_2016.merge(votos_2016, on="SQ_CANDIDATO", how="inner")
cand_votos_2020 = cand_2020.merge(votos_2020, on="SQ_CANDIDATO", how="inner")

In [1015]:
print(f"Dimensões após integração - 2012: {cand_votos_2012.shape}")
print(f"Dimensões após integração - 2016: {cand_votos_2016.shape}")
print(f"Dimensões após integração - 2020: {cand_votos_2020.shape}")

Dimensões após integração - 2012: (576, 14)
Dimensões após integração - 2016: (560, 14)
Dimensões após integração - 2020: (845, 14)


In [1016]:
# % de Votos Nominais: Calcule a porcentagem de votos nominais de cada candidato em relação ao total de votos.
for df in [cand_votos_2012, cand_votos_2016, cand_votos_2020]:
    total_votos = df["QT_VOTOS_NOMINAIS"].sum()
    df["PERC_VOTOS"] = (df["QT_VOTOS_NOMINAIS"] / total_votos) * 100

In [1017]:
# Classificação por Desempenho
for df in [cand_votos_2012, cand_votos_2016, cand_votos_2020]:
    df["CLASSIFICACAO"] = df["QT_VOTOS_NOMINAIS"].rank(ascending=False, method="min").astype(int)

In [1018]:
# Situação Eleitoral
for df in [cand_votos_2012, cand_votos_2016, cand_votos_2020]:
    df["SITUACAO"] = df["CLASSIFICACAO"].apply(lambda x: "ELEITO" if x == 1 else "NÃO ELEITO")


### 1.2 Dataframe final

In [1019]:
# Adicionando a coluna do ano
cand_votos_2012["ANO_ELEICAO"] = 2012
cand_votos_2016["ANO_ELEICAO"] = 2016
cand_votos_2020["ANO_ELEICAO"] = 2020

# Unindo os DataFrames
cand_votos_total = pd.concat([cand_votos_2012, cand_votos_2016, cand_votos_2020], ignore_index=True)

# Exibindo dimensões do DataFrame unificado
print(f"Dimensões do DataFrame consolidado: {cand_votos_total.shape}")

Dimensões do DataFrame consolidado: (1981, 17)


In [1020]:
# Salvando o DataFrame consolidado
cand_votos_total.to_csv("dados_tratados/dataframe_final.csv", index=False, encoding="utf-8")
print("Arquivo consolidado salvo com sucesso!")

Arquivo consolidado salvo com sucesso!


# 4. Análise exploratória e visualizações.

# 5. Preparação do dataset final para análise.